In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('911_Police_Calls_for_Service.csv', sep=';')

In [6]:
df.head()

,recordId,callDateTime,priority,district,description,callNumber,incidentLocation,location
0,1423624,05/04/2016 09:58:00 PM,High,ND,SILENT ALARM,P161253035,400 WINSTON AV,"400 WINSTON AV\nBALTIMORE, MD\n(39.349792, -76..."
1,1402097,04/27/2016 03:57:00 PM,Medium,SW,911/HANGUP,P161182081,1400 BRADDISH AV,"1400 BRADDISH AV\nBALTIMORE, MD\n(39.303941, -..."
2,1420176,05/03/2016 06:40:00 PM,Medium,ED,DISORDERLY,P161242705,200 E NORTH AV,"200 E NORTH AV\nBALTIMORE, MD\n(39.311294, -76..."
3,1423653,05/04/2016 10:10:00 PM,Medium,NE,911/NO VOICE,P161253068,2500-1 HARFORD RD,"2500 1 HARFORD RD\nBALTIMORE, MD\n(39.316763, ..."
4,1417949,05/03/2016 12:29:00 AM,Non-Emergency,SD,Private Tow,P161240063,100 W PATAPSCO AV,"100 W PATAPSCO AV\nBALTIMORE, MD\n(39.239215, ..."


In [11]:
df.priority.fillna('NaN', inplace=True)
df.description.fillna('NaN', inplace=True)

In [13]:
df.shape

(3242420, 8)

In [14]:
priority = df.priority.value_counts().to_frame()
priority.priority.sum()

3242420

In [16]:
description = df.description.value_counts().to_frame()

In [18]:
description['percentage'] = description.description/description.description.sum()

In [21]:
# description.to_csv('tmp-911-description.csv')
description.shape

(8810, 2)